## Discrete Multivariate Distributions

Multivariate distributions describe the probabilistic behavior of *multiple* quantities and their relationships. Every Statistical and Machine Learning method involves multiple random variables (RVs) and their distributions: joint,  conditionals, and margninals. In fact, the foundation of each method lies on how it models the dependence structure of the RVs, which is typically done in way that facilitates analysis and/or computation. In other words, different models make assumptions about how the variables are related to allow for efficient processing. Experienced Data Scientists understand the modelling assumptions and their limitations, allowing them to choose and improve appropriate methods for the problem at hand.

In this notebook we build a [Naive Bayes Classifier (NBC)](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) for spam detection, one of the earliest and most widespread applications of [Statistical classification](https://en.wikipedia.org/wiki/Statistical_classification). Although the probability model for NBC is very simple, the method works extremely well for email filtering and other text classification. We will first look at the model details, and then we will implement the NBC in Python and compare its results with the built-in function in [scikit-learn](https://scikit-learn.org/stable/), Python's principal Machine Learning library.


### Naive Bayes Classification

Classification is the problem of *predicting the class* of an object based on its other observed characteristics which are  probabilistically related to its class. We denote the class by the *categorical* (i.e., discrete finite) RV $Y$, called the *response* or *label*, and we denote the related variables by $X_1,\ldots,X_p$, called the *predictors* of *features*. For out purposes we will assume the predictors are also discrete finite RVs. So, to summarize, the goal of NBC is to predict the (unknown) value of $Y$ based on the (observed) values of $X_1,\ldots, X_p$. 

##### Maximum A Posteriori Classification

Assume we know the *joint* distribution of all #$(p+1)$ RVs $Y,X_1,\ldots, X_p$ for starters. If we observed the values $x_1,\ldots, x_p$ for $X_1,\ldots, X_p$, our best description of $Y$ would be the *conditional* distribution:
$$ P( Y = y | X_1 =x_1, \ldots, X_p = x_p ),\quad \text{ for }  y \in \text{ range }(Y) $$
which gives you the conditional probabilities of the possible values of $Y$. If we then had to *guess* which value the RV $Y$ takes, we would choose the *most likely* one, i.e. that which maximizes the conditional probability:
$$ \hat{y} =  \arg \max_y P( Y = y | X_1 =x_1, \ldots, X_p = x_p ) $$
This approach is called [Maximum a Posteriori (MAP)](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation) prediction, from the name of the conditional distribution, which is called the [*posterior*](https://en.wikipedia.org/wiki/Posterior_probability) in this context. 


Furthermore, from Bayes theorem we get:
$$ P(Y=y|X_1=x_1,\ldots,X_p=x_p) = \frac{  P(Y=y, X_1 = x_1, \ldots, X_p = x_p ) } { P( X_1 = x_1, \ldots, X_p = x_p ) } \\ = \frac{  P( X_1 = x_1, \ldots, X_p = x_p | Y=y ) P(Y=y) } { P( X_1 = x_1, \ldots, X_p = x_p ) } $$
Note that $y$ appears only in the numerator, so maximizing the posterior is equivalent to maximizing the numerator, which is essentially a re-expression of the *joint* distribution. This is the form of the maximization we will perform, in order to avoid unnecessary calculations. 

Also note that *if* the predictor RVs $X_1,\ldots, X_p$ where *independent* of the response RV $Y$, then its conditional/posterior distribution would be the same as its *marginal* distribution:
$$P( Y = y | X_1 =x_1, \ldots, X_p = x_p ) = P( Y = y)$$
Intuitively, independence implies that the information obtained from $X_1, \ldots, X_p$ is irrelevant for predicting $Y$. When building a classifier, we would therefore want to use predictors that are as *dependent* as possible to the response. As an extreme example, if you had a predictor $X$ that was *perfectly* related to $Y$, i.e. there is a 1-to-1 relation between the values of $X$ and $Y$, then knowing the value of $X$ would tell you the value of $Y$.




##### Naive Bayes Assumption

We have seen how Bayesian classification works, at least conceptually, but to actually *apply* it we need a workable form of the *joint* distribution of $Y,X_1,\ldots, X_p$. Since all variables are finite, their joint distribution can be represented as a $(1+p)$-dimensional array. In Statistical/Machine Learning applications, the probability model is not provided, but has to be *estimated/trained* based on data. E.g., assuming all variables are *binary*, there are around $2^{(p+1)}$ probabilities to estimate in the joint distribution array. Because the number of parameters increases *exponentially* in the number of dimensions/predictors, good estimation quickly becomes infeasible (requires astronomical amounts of data); what is called the [curse of dimensionality](link here). 

To solve this problem, NBC makes a *naive* (not reallistic) simplifying assumption: it assumes features are *conditionally independent* given the class $Y$. In practice, this implies that:
$$ P( X_1 = x_1, \ldots, X_p = x_p | Y=y ) P(Y=y) \\
= P(X_1 = x_1 | Y = y )  \times \cdots \times P(X_p = x_p | Y = y ) \times P( Y = y )  \\
= \left( \prod_{i=1}^{p} P(X_i = x_i | Y = y ) \right)  \times P( Y = y ) $$
I.e., the joint distribution can be expressed as a product of the $X$'s 1D conditionals times $Y$'s 1D marginal. By imposing this special structure we effectively reduce the dimensionality of the problem: instead of one $(p+1)$-dimensional distribution, we work with $(p+1)$ 1-dimensional distributions. E.g., if we assume RVs are binary, there are $2(p+1)$ probabilities to estimate, rather than $2^{(p+1)}-1$ for the general model. Now that we have know the main idea behind NBC, we look at a specific application for spam detection.


##### Spam Data  

We will be working with a subset of the [Enron-Spam dataset](http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/index.html) consisting of 5726 emails, 1368 of which (23.9%) are *spam* and the remaining 4358 *ham* (legitimate). 
We use the [pandas](https://pandas.pydata.org/) library in Python to work with table/spreadsheet-like data, where each row represents an observation and each column a field/variable. Column ```text``` contains the email text, and column ```spam``` indicates if the email is spam (1) or not (0); below is a preview:

In [1]:
import pandas as pd
emails = pd.read_csv("./data/emails.csv",  usecols=[0, 1])
emails.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words = 'english', binary = 'True', min_df=10, max_df = .30) # initialize vectorizer
X = vectorizer.fit_transform( emails.text )   # apply vectorizer to emails text column; output is sparse binary matrix 
X_words = vectorizer.get_feature_names_out()  # extract words
Y = emails.spam                               # extract response

import random; random.seed(1)
random.sample( list(X_words), 10)

['capture',
 'reaching',
 'aiesec',
 'enclosed',
 'bother',
 'ordered',
 'measures',
 'narrow',
 'spark',
 'introduced']

##### Probability Estimation

We now estimate the joint distribution, which for NBC is given by the conditional probabilities of each word being present in a spam or non-spam email, i.e. $P( X_i=1 | Y=1 )$ and $P( X_i = 1 | Y=0)$ respectively. Notice that for any binary feature we have $ P( X_i=0 | Y=y ) = 1 - P( X_i=1 | Y=y)$ for any $y=0,1$ by the complement rule, i.e. just one probability value determines the conditional distribution. 

Let's focus on $ P( X_i=1 | Y=1 ) = \frac{P( X_i=0, Y=1 ) }{P( Y=1 )}$: we could estimate this probability based on data by dividing the number of times word $i$ appears in spam emails (corresponding to $ \{X_i=0\} \cap \{Y=1\}$) by the number of spam emails ($\{Y=1\}$) in our data. But this approach has a problem: if there are no spam emails with that word in our data, the estimated probability will be 0, which implies it is *impossible* to ever have a spam email with that word. To avoid such extremes we employ [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), adjusting the ratio to allow a small probability of observing a word in every type of email. The estimated conditional probabilities become:

$$ 
\hat{P}( X_i=1 | Y=1 ) = \frac{ \text{ # spam emails w/ word $i$}+1 }{ \text{# spam emails}+2 } \\
\hat{P}( X_i=1 | Y=0 ) = \frac{ \text{ # ham emails w/ word $i$}+1 }{ \text{# ham emails}+2 } 
$$
(the *hat* notation $\hat{P}$ indicates this is an estimate). This, combined with the estimated (marginal) probability of spam:
$$ \hat{P}(Y=1) = \frac{ \text{ # spam emails } } { \text{ # emails } } $$
is all we need for NBC.


In [3]:
import numpy as np
P_X_Y1 = ( np.sum( X[ Y == 1 ] , axis = 0) + 1 ) / ( sum( Y == 1 ) + 2 ) # Estimate P( X_i=1 | Y=1 )
P_X_Y0 = ( np.sum( X[ Y == 0 ] , axis = 0) + 1 ) / ( sum( Y == 0 ) + 2 ) # Estimate P( X_i=1 | Y=0 )
P_Y1 = sum( Y == 1 ) / len(Y)  # Estimate P( Y=1 )
P_Y0 = 1 - P_Y1

# np.sum( X[ Y == 1 ] , axis = 0) returns a vector with word counts:
# X[ Y==1 ] uses Boolean array indexing to extract spam (Y==1) rows
# (https://numpy.org/doc/stable/user/basics.indexing.html)
# and np.sum( . , axis = 0 ) sums along word feature columns 

##### Classification

With probabilities estimated, we now apply our classifier on the same data to evualuate its performance. Using only the word features ($X$'s), we compare the two conditional probabilities $P( Y=1 | X_1=x_1,\ldots,X_p=x_p)$ vs $P( Y=0 | X_1=x_1,\ldots,X_p=x_p)$ and choose the class ($Y$=0/ham or 1/spam) giving the highest one. We saw above that we only need to calculate the numerator (denominator is common), which by the Naive Bayes assumption is factorized as:
$$ P( X_1 = x_1, \ldots, X_p = x_p | Y=y ) P(Y=y) =  \prod_{i=1}^{p} P(X_i = x_i | Y = y ) \times P( Y = y ) $$
If you tried to calculate this product in Python you would get 0 because it multiplies thousands of small probabilities. For numerical stability, we will compare the *logarithms* of the products which are the sums of their logs:
$$ \log  \left( \prod_{i=1}^{p} P(X_i = x_i | Y = y ) \times P( Y = y ) \right) = \sum_{i=1}^{p} \log ( P(X_i = x_i | Y = y ) ) + \log(  P( Y = y ) ) $$
The following code loops over every email, compares the log-probabilities for ham/spam, and selects the class with the highest one. Note the use of the complement rule $P(X_i=0|Y=y) = 1 - P(X_i=1|Y=y)$ for features with $X_i = 0$. At the end, it calculates the *accuracy*, i.e. the proportion of correctly classified emails, which is an impressive 96.75%!

In [4]:
Y_my_pred = np.zeros(len(Y))

for i in np.arange(0, len(Y)) :
    lP_Y0X = np.sum( np.log( P_X_Y0[ X[i].todense() == 1 ] ) ) + np.sum( np.log( 1 - P_X_Y0[ X[i].todense() == 0 ] ) ) + np.log( P_Y0 )
    lP_Y1X = np.sum( np.log( P_X_Y1[ X[i].todense() == 1 ] ) ) + np.sum( np.log( 1 - P_X_Y1[ X[i].todense() == 0 ] ) ) + np.log( P_Y1 )
    if lP_Y1X > lP_Y0X:
        Y_my_pred[i] = 1

sum( Y_my_pred == Y ) / len(Y)   # caclulate accuracy (% correct classification)

0.9675165909884736

We have effectively implemented the same procedure used in ```scikit-learn```, albeit less efficiently. For comparison, the following code gives 

<1018x6233 sparse matrix of type '<class 'numpy.int64'>'
	with 66870 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()      # initiate Bernoulli (= binary feature) NBC 
clf.fit(X, Y)            # train model 
Y_pred = clf.predict(X)  # extract predictions
sum( Y_pred == Y ) / len(Y) #

0.9675165909884736

In [6]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, Y_pred) )

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))


NameError: name 'Y_test' is not defined

In [288]:
sum( y_train == 1)



0.7629250116441546 0.23707498835584537 1.0


### Problems


Before applying the classifier, we split our data into a [training and test](https://en.wikipedia.org/wiki/Training,_validation,_and_test_data_sets) set; the training set is used to estimate the probabilities, and the test is used to evaluate the performance of the procedure. We do this to get an *objective* measure of performance on data that were not used to set up 



In [13]:

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.25, random_state=1234)

clf = BernoulliNB()
clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)
sum( Y_pred == Y_test ) / len(Y_test)


from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, Y_pred) )

from sklearn.metrics import classification_report
print(classification_report(Y_test, Y_pred))



0.9629888268156425